# CMT2idagrn

Reads GCMT events and builds evt_######## files which are input to idagrn to make synthetic seismograms

##### JBR - 2/7/18

In [13]:
%load_ext autoreload
%autoreload
from setup_parameters import *
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.core import AttribDict
from obspy.io.sac import SACTrace
from obspy import read
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from datetime import datetime
import os
import calendar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Load Client
client = Client(webservice)
print(client)

if not os.path.exists(CMT2idagrn_path):
    os.makedirs(CMT2idagrn_path)

FDSN Webservice Client (base url: http://service.iris.edu)
Available Services: 'dataselect' (v1.1.5), 'event' (v1.1.8), 'station' (v1.1.32), 'available_event_catalogs', 'available_event_contributors'

Use e.g. client.help('dataselect') for the
parameter description of the individual services
or client.help() for parameter description of
all webservices.


In [15]:
t1 = UTCDateTime(tstart)
t2 = UTCDateTime(tend)

# Load IRIS catalogue to get events of interest
catIRIS = client.get_events(starttime=t1, endtime=t2, minmagnitude=minmagnitude)
print(catIRIS.__str__(print_all=True))

11 Event(s) in Catalog:
2015-02-27T13:45:05.640000Z |  -7.358, +122.491 | 7.0 MW
2015-02-13T18:59:13.510000Z | +52.510,  -32.021 | 7.1 MW
2014-11-15T02:31:42.240000Z |  +1.834, +126.463 | 7.0 MW
2014-11-09T14:38:15.600000Z | +46.930, +140.630 | 7.6 mb
2014-11-01T18:57:22.350000Z | -19.802, -177.833 | 7.1 MW
2014-10-14T03:51:37.110000Z | +12.589,  -88.072 | 7.3 MW
2014-10-09T02:14:29.740000Z | -32.299, -110.924 | 7.0 MW
2014-06-23T20:53:09.930000Z | +51.703, +178.643 | 7.9 MW
2014-04-19T13:28:00.720000Z |  -6.708, +154.992 | 7.5 MW
2014-04-18T14:27:24.690000Z | +17.321, -100.994 | 7.3 MW
2014-04-13T12:36:19.050000Z | -11.467, +161.960 | 7.4 MW


In [16]:
for iev in range(0,len(catIRIS)) : 
    # extract date for naming folders
    time = catIRIS[iev].origins[0].time
    date = datetime.strptime(str(time),'%Y-%m-%dT%H:%M:%S.%fZ')
    evname = date.strftime('%Y%m%d%H%M')
    month = calendar.month_abbr[date.month].lower()
    year = str(date.year)
    time1 = str(time-50)
    time2 = str(time+50)
    catCMT = obspy.read_events("https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/"+year+"/"+month+""+year[2:4]+".ndk")
    CMT_ev = catCMT.filter("time > "+time1,
                                "time < "+time2,
                                "magnitude >= "+str(minmagnitude-1))
    if len(CMT_ev)==0:
        print("No focal mechanism for "+evname)
        continue
    
    const = 1e19
    scale = 0.1
    lat = CMT_ev[0].origins[1].latitude
    lon = CMT_ev[0].origins[1].longitude
    depth = CMT_ev[0].origins[1].depth
    m_rr = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_rr/const
    m_tt = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_tt/const
    m_pp = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_pp/const
    m_rt = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_rt/const
    m_rp = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_rp/const
    m_tp = CMT_ev[0].focal_mechanisms[0].moment_tensor.tensor.m_tp/const
    
    print('Working on '+CMT2idagrn_path+'evt_'+evname)
    f = open(CMT2idagrn_path+'evt_'+evname,'w')
    f.write("%s\n%.3f %.3f %.2f\n%.1f\n%.5f\n%.5f\n%.5f\n%.5f\n%.5f\n%.5f\n" % (evname,lat,lon,depth/1000,scale,m_rr,m_tt,m_pp,m_rt,m_rp,m_tp))
    f.close()
    
    
    

Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201502271345
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201502131859
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201411150231
No focal mechanism for 201411091438
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201411011857
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201410140351
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201410090214
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201406232053
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201404191328
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201404181427
Working on ./IRIS_YO_7.0/CMT2idagrn/evt_201404131236
